In [26]:
from explainability.structured.core.structured_challenger import SKChallenger
from explainability.structured.samples.challengers.categorize_challenger import CategorizeChallenger
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [36]:
# Load adult income dataset, 100 rows
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None, nrows=1000)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry",
    "Income"
]
df.drop([
    "fnlwgt", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race",
    "NativeCountry"
], axis=1, inplace=True)
label_column = "Income"

In [37]:
# Define our base scikit-learn model
model = DecisionTreeClassifier(max_depth=3)

# Sample Challenger

In [38]:
cc = CategorizeChallenger(DecisionTreeClassifier(max_depth=3),
                          df, label_column)

In [39]:
# creates multiple structured manipulators, one for each predefined challenge
cc.generate_challenges()

In [40]:
# base contains the original dataset, which is always included in a challenge
cc.challenges["base"].df.head()

,Age,WorkClass,Education,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Income
0,52,?,12th,Male,594,0,40,<=50K
1,29,Private,HS-grad,Male,0,0,40,<=50K
2,44,Private,HS-grad,Female,0,0,40,<=50K
3,37,Private,Bachelors,Male,0,0,60,>50K
4,33,Private,Some-college,Female,0,0,40,<=50K


In [41]:
# an example of a challenge dataset
cc.challenges["challenge_2"].df.head()

,Age,WorkClass,Education,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Income
0,23,Private,HS-grad,Female,0,0,"(33.667, 66.333]",<=50K
1,24,Self-emp-inc,Bachelors,Male,0,0,"(33.667, 66.333]",<=50K
2,24,Private,HS-grad,Male,0,0,"(33.667, 66.333]",<=50K
3,23,Private,HS-grad,Male,0,0,"(33.667, 66.333]",<=50K
4,28,Private,HS-grad,Male,0,0,"(33.667, 66.333]",<=50K


In [42]:
# trains one of our models for each challenge
cc.train_models()

In [44]:
# Evaluate base model
X_train, y_train, X_test, y_test = cc.challenges["base"].train_test_split()
cc.models["base"].score(X_test, y_test)

0.82

In [45]:
# Evaluate a challenge model
_, _, X_test, y_test = cc.challenges["challenge_2"].train_test_split()
cc.models["challenge_2"].score(X_test, y_test)

0.845

# Custom Challenger